<a href="https://colab.research.google.com/github/MakC-Ukr/AP_Project_Group134/blob/master/Week9/comibining_NER_negation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import spacy
from nltk.corpus import stopwords
import nltk
import re
from nltk.stem import WordNetLemmatizer 
nltk.download('stopwords')
nltk.download('punkt')
# from flair.data import Sentence
# from flair.models import SequenceTagger
nlp = spacy.load("en_core_web_sm")
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("drive/My Drive/Extramarks Project/Dataset versions/Model1-Semantic/Jacc_sim_ignore_syllabus.csv")
df = df.drop("Unnamed: 0", axis = 1)

In [ ]:
df

,syllabus_id1,syllabus_id2,ques1,ques2,question_id1,question_id2
0,700295,700313,how many π bonds are present in ferrocene,how many π are present in ferrocene,2073346,2073271
1,700295,731162,how many π bonds are present in ferrocene,how many π bonds are present in silicon dioxide,2073346,1909343
2,698013,697895,what is covalency of chlorine atom in second e...,what is covalency of chlorine atom in second e...,2073324,2073165
3,697895,697895,calculate value of x y here x and y are the to...,calculate value of x y here x and y are the to...,2073215,2073343
4,697895,698013,in phosphorus acid if x is number of non bondi...,in phosphorus acid if x is number of non bondi...,2073202,2073272
...,...,...,...,...,...,...
6639,511206,511067,tim said that he,tim said that he,1551779,1451802
6640,511206,4770,tim said that he,he said that,1551779,494476
6641,511206,4770,tim said that he,he said that,1551779,494307
6642,511206,511067,you look so shabby you,you look so shabby you,1551630,1451801


In [ ]:
non_dupl = []
stops = set(['a', 'the', 'an', 's'])
# s added for cases like henry's henry s

for index, row in df.iterrows():
  flag = True

  if index % 1000 == 0:
    print(index)

  word_list_1 = nltk.word_tokenize(row['ques1'])
  word_list_1 = [word if word not in stops else '' for word in word_list_1]
  lemmatized_output_1 = ' '.join([lemmatizer.lemmatize(w) for w in word_list_1])
  lemmatized_output_1 = re.sub(' +', ' ', lemmatized_output_1)
  lemmatized_output_1 = re.sub("'s", '', lemmatized_output_1)

  word_list_2 = nltk.word_tokenize(row['ques2'])
  word_list_2 = [word if word not in stops else '' for word in word_list_2]
  lemmatized_output_2 = ' '.join([lemmatizer.lemmatize(w) for w in word_list_2])
  lemmatized_output_2 = re.sub(' +', ' ', lemmatized_output_2)
  lemmatized_output_2 = re.sub("'s", '', lemmatized_output_2)


  ques1 = nlp(lemmatized_output_1)
  ques2 = nlp(lemmatized_output_2)


  words_1 = {}
  words_2 = {}

  for entity in ques1.ents:
    words_1[entity.text] = entity.label_
  
  for entity in ques2.ents:
    words_2[entity.text] = entity.label_
    if entity.text not in words_1:
      non_dupl_row = pd.Series(row)
      non_dupl_row['diff_entity'] = entity.text # this is partly wrong since multiple entities can differ
      non_dupl_row['diff_entity_tag'] = entity.label_ # this is partly wrong since multiple entities can differ
      # non_dupl.append(non_dupl_row)
      flag = False
      break

  for entity in ques1.ents:
    if entity.text not in words_2:
      non_dupl_row = pd.Series(row)
      non_dupl_row['diff_entity'] = entity.text # this is partly wrong since multiple entities can differ
      non_dupl_row['diff_entity_tag'] = entity.label_ # this is partly wrong since multiple entities can differ
      # non_dupl.append(non_dupl_row)
      flag = False
      break

  non_dupl.append(flag)

0
1000
2000
3000
4000
5000
6000


In [ ]:
def calc_curr_to_next(words_ls):
  res = defaultdict(str)
  for index, word in enumerate(words_ls):
    res[word] = words_ls[index+1] if index+1 < len(words_ls) else "*"
  return res

def generate_list_of_neg_affected_words(ls, contractions, window_size):
  neg_1 = []

  for index in range(len(ls)):
    word = ls[index]
    if word in contractions:
      for j in range(window_size):
        neg_1.append(True)
      index+= (window_size-1)
    else:
      neg_1.append(False)
    
  return neg_1


WINDOW_SIZE = 3

def check_pair_negation(ques1, ques2):
  contractions = ["not", "doesn't", "couldn't", "can't", "wouldn't", "shouldn't", "haven't", "hasn't", "hadn't"]
  wrong_spellings = {"cannot":"can't", "doesnot":"doesn't", "couldnot":"couldn't", "canot":"can't", "wouldnot":"wouldn't", "shouldnot":"shouldn't", "havenot":"haven't", "hasnot":"hasn't", "hadnot":"hadn't"}
  articles = set(['a', 'the', 'an'])


  w1 = ques1.lower().split()
  w2 = ques2.lower().split()

  w1 = [w for w in w1 if w not in articles]
  w2 = [w for w in w2 if w not in articles]

  w1 = [wrong_spellings[w] if w in wrong_spellings else w for w in w1]
  w2 = [wrong_spellings[w] if w in wrong_spellings else w for w in w2]

  words_1 = set(w1)
  words_2 = set(w2)

  C2N_1 = calc_curr_to_next(w1) 
  C2N_2 = calc_curr_to_next(w2) 

  for not_w in contractions:
    if not_w in words_1 and not_w not in words_2 or not_w in words_2 and not_w not in words_1:
      return True
    elif not_w in words_1 and C2N_1[not_w] != C2N_2[not_w]: ## skeptical
      return True
    elif not_w in words_2 and C2N_2[not_w] != C2N_1[not_w]:
      return True

  # V2
  neg_1 = generate_list_of_neg_affected_words(w1, contractions, WINDOW_SIZE)
  neg_2 = generate_list_of_neg_affected_words(w2, contractions, WINDOW_SIZE)

  was_negated_1 = defaultdict(int)
  was_negated_2 = defaultdict(int)

  for index, w in enumerate(w1):
    if neg_1[index]:
      was_negated_1[w] = was_negated_1[w]+ 1

  for index, w in enumerate(w2):
    if neg_2[index]:
      was_negated_2[w] = was_negated_2[w]+ 1

  keys_both = list(was_negated_1.keys())
  keys_both.extend(list(was_negated_2.keys()))

  for key in keys_both:
    if was_negated_1[key] != was_negated_2[key]:
      True


  return False


to_be_appended = []

for index, row in df.iterrows():
  if check_pair_negation(row['ques1'], row['ques2']):
    to_be_appended.append(False)
  else:
    to_be_appended.append(True)

In [ ]:
pd.Series(to_be_appended).value_counts()

True     6395
False     249
dtype: int64

In [ ]:
pd.Series(non_dupl).value_counts()

True     6370
False     274
dtype: int64

In [ ]:
combined = (pd.Series(non_dupl) * pd.Series(to_be_appended))
combined.value_counts()

True     6122
False     522
dtype: int64

In [ ]:
newDf = df[combined]
newDf.to_csv("drive/My Drive/Extramarks Project/Dataset versions/Combined_NER_Negation.csv")
newDf

,syllabus_id1,syllabus_id2,ques1,ques2,question_id1,question_id2
0,700295,700313,how many π bonds are present in ferrocene,how many π are present in ferrocene,2073346,2073271
1,700295,731162,how many π bonds are present in ferrocene,how many π bonds are present in silicon dioxide,2073346,1909343
2,698013,697895,what is covalency of chlorine atom in second e...,what is covalency of chlorine atom in second e...,2073324,2073165
3,697895,697895,calculate value of x y here x and y are the to...,calculate value of x y here x and y are the to...,2073215,2073343
4,697895,698013,in phosphorus acid if x is number of non bondi...,in phosphorus acid if x is number of non bondi...,2073202,2073272
...,...,...,...,...,...,...
6637,511206,453,i thought that bus fares,i thought that the bus fares,1552424,1300480
6638,511206,4765,i thought that bus fares,i thought that the bus fares,1552424,494809
6639,511206,511067,tim said that he,tim said that he,1551779,1451802
6642,511206,511067,you look so shabby you,you look so shabby you,1551630,1451801
